In [66]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk

# CARREGANDO BASE DE DADOS

In [67]:
dados = pd.read_csv("stroke.csv")
dados.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


# ANALISE GERAL DADOS DADOS

In [68]:
dados.dtypes

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [69]:
print(dados.isnull().sum())
dados[dados.isnull().any(axis=1)]

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
13,8213,Male,78.0,0,1,Yes,Private,Urban,219.84,NaN,Unknown,1
19,25226,Male,57.0,0,1,No,Govt_job,Urban,217.08,NaN,Unknown,1
27,61843,Male,58.0,0,0,Yes,Private,Rural,189.84,NaN,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5039,42007,Male,41.0,0,0,No,Private,Rural,70.15,NaN,formerly smoked,0
5048,28788,Male,40.0,0,0,Yes,Private,Urban,191.15,NaN,smokes,0
5093,32235,Female,45.0,1,0,Yes,Govt_job,Rural,95.02,NaN,smokes,0
5099,7293,Male,40.0,0,0,Yes,Private,Rural,83.94,NaN,smokes,0


In [70]:
dados.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [71]:
for coluna in dados.columns:
    if dados[coluna].dtype == 'object':
        print(f"\nColuna '{coluna}':\nTipos possíveis: {dados[coluna].unique()}")


Coluna 'gender':
Tipos possíveis: ['Male' 'Female' 'Other']

Coluna 'ever_married':
Tipos possíveis: ['Yes' 'No']

Coluna 'work_type':
Tipos possíveis: ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']

Coluna 'Residence_type':
Tipos possíveis: ['Urban' 'Rural']

Coluna 'smoking_status':
Tipos possíveis: ['formerly smoked' 'never smoked' 'smokes' 'Unknown']


# PRE-PROCESSAMENTO DOS DADOS

1. Remoção de linhas nulas
2. Separação dos dados em X (variáveis independentes) e y (variável dependente)
3. Transformação das variáveis categóricas em numéricas (LabelEncoder e OneHotEncoder)
4. Separação em Teste e Treino
5. Normalização

# REMOVENDO DADOS NULOS

In [72]:
dados=dados.dropna()
dados[dados.isnull().any(axis=1)]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke


# SEPARANDO EM X E Y | PREVISORES E CLASSE | INDEPENDENTE E DEPENDENTE

In [73]:
x = dados.iloc[:, 1:-1]
y = dados.iloc[:, -1]
x.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked


# TRANSFORMANDO VARIAVEIS CATEGORICAS BINARIAS EM NUMERICAS

In [74]:
from sklearn.preprocessing import LabelEncoder

colunas = ['ever_married', 'Residence_type'] 
label = LabelEncoder()
x[colunas] = x[colunas].apply(label.fit_transform)

x.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,67.0,0,1,1,Private,1,228.69,36.6,formerly smoked
2,Male,80.0,0,1,1,Private,0,105.92,32.5,never smoked
3,Female,49.0,0,0,1,Private,1,171.23,34.4,smokes
4,Female,79.0,1,0,1,Self-employed,0,174.12,24.0,never smoked
5,Male,81.0,0,0,1,Private,1,186.21,29.0,formerly smoked


# TRANSFORMADOS DADOS CAETAGORICOS COM MAIS DE DUAS OPÇÃO

In [75]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder 

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['gender', 'work_type', 'smoking_status'])], 
                       remainder='passthrough')
x = ct.fit_transform(x)

x[0]

array([  0.  ,   1.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,   0.  ,
         0.  ,   1.  ,   0.  ,   0.  ,  67.  ,   0.  ,   1.  ,   1.  ,
         1.  , 228.69,  36.6 ])

# AJUSTANDO O NOME DAS COLUNAS

In [76]:
colunas = ['ohe1', 'ohe2', 'ohe3', 'ohe4', 'ohe5', 'ohe6', 'ohe7', 'ohe8', 'ohe9', 'ohe10', 'ohe11', 
           'ohe12', 'age', 'hypertension', 'heart_disease', 'ever_married', 'Residence_type', 
           'avg_glucose_level', 'bmi']
pd.DataFrame(x, columns=colunas)

,ohe1,ohe2,ohe3,ohe4,ohe5,ohe6,ohe7,ohe8,ohe9,ohe10,ohe11,ohe12,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,67.0,0.0,1.0,1.0,1.0,228.69,36.6
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,80.0,0.0,1.0,1.0,0.0,105.92,32.5
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49.0,0.0,0.0,1.0,1.0,171.23,34.4
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,79.0,1.0,0.0,1.0,0.0,174.12,24.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,81.0,0.0,0.0,1.0,1.0,186.21,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,103.08,18.6
4905,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,81.0,0.0,0.0,1.0,1.0,125.20,40.0
4906,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,35.0,0.0,0.0,1.0,0.0,82.99,30.6
4907,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,51.0,0.0,0.0,1.0,0.0,166.29,25.6


# SEPARANDO TREINO E TESTE

In [77]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)
print(len(x_train), len(x_test))

4418 491


# NORMALIZANDO OS DADOS PREVISORES

In [78]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train[:3, :]

array([[-1.20374508,  1.20430914, -0.01504653,  2.61939114, -0.07074284,
        -1.16135725, -0.43021604, -0.3990937 , -0.65698303, -0.45182167,
        -0.77881028,  2.36725292,  0.76768419,  3.12659715, -0.22617236,
         0.72785877, -1.01505216, -0.11269978, -0.05480295],
       [-1.20374508,  1.20430914, -0.01504653, -0.38176811, -0.07074284,
        -1.16135725, -0.43021604,  2.50567724, -0.65698303, -0.45182167,
         1.28400976, -0.42243057, -1.32256567, -0.31983654, -0.22617236,
        -1.3738929 , -1.01505216,  0.0706691 , -1.46011864],
       [-1.20374508,  1.20430914, -0.01504653, -0.38176811, -0.07074284,
         0.86106149, -0.43021604, -0.3990937 ,  1.52210933, -0.45182167,
        -0.77881028, -0.42243057, -1.14467206,  3.12659715, -0.22617236,
        -1.3738929 , -1.01505216, -0.98172846,  8.6936037 ]])

In [79]:
y_train

4338    0
2146    0
2128    0
3782    0
349     0
       ..
533     0
4081    0
423     0
795     0
2794    0
Name: stroke, Length: 4418, dtype: int64

# CRIANDO A ARQUETETURA DA REDE NEURAL (MLP)

In [88]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier

In [81]:
def create_model(optimizer='adam'):
    modelo = Sequential()
    modelo.add(Dense(units=6, activation='relu', kernel_initializer='he_normal'))
    modelo.add(Dense(units=6, activation='relu', kernel_initializer='he_normal'))
    modelo.add(Dense(units=1, activation='sigmoid'))
    modelo.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return modelo

# CRIANDO GridSearch

In [82]:
modelo = KerasClassifier(model=create_model, verbose=2)

In [83]:
optimizer = ['SGD', 'Adam']
batch_size = [16, 32, 64]
epochs = [10, 20, 30, 40, 50]

param_grid = dict(
    optimizer=optimizer, 
    batch_size=batch_size, 
    epochs=epochs)

# NÃ TO CONSEGUINDO ARRUMAR O ERRO

In [84]:
grid = GridSearchCV(estimator=modelo, param_grid=param_grid, n_jobs=None, cv=5)
grid_result = grid.fit(x_train, y_train)

ValueError: 
All the 150 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\GABRIEL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\GABRIEL\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "c:\Users\GABRIEL\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py", line 760, in fit
    self._fit(
  File "c:\Users\GABRIEL\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py", line 926, 

# Resultados

In [ ]:
print(f"Melhor acurácia: {grid_result.best_score_}")
print(f"Melhores parâmetros: {grid_result.best_params_}")

for mean, std, params in zip(grid_result.cv_results_['mean_test_score'], 
                              grid_result.cv_results_['std_test_score'], 
                              grid_result.cv_results_['params']):
    print(f"Acurácia média: {mean:.4f} (Desvio padrão: {std:.4f}) com parâmetros: {params}")


Best: 0.956315 using {'batch_size': 32, 'epochs': 40, 'optimizer': 'SGD'}
0.956089 (0.004658) with: {'batch_size': 16, 'epochs': 10, 'optimizer': 'SGD'}
0.955637 (0.004545) with: {'batch_size': 16, 'epochs': 10, 'optimizer': 'Adam'}
0.956089 (0.004658) with: {'batch_size': 16, 'epochs': 20, 'optimizer': 'SGD'}
0.955410 (0.004778) with: {'batch_size': 16, 'epochs': 20, 'optimizer': 'Adam'}
0.956089 (0.004658) with: {'batch_size': 16, 'epochs': 30, 'optimizer': 'SGD'}
0.955863 (0.004744) with: {'batch_size': 16, 'epochs': 30, 'optimizer': 'Adam'}
0.956089 (0.004658) with: {'batch_size': 16, 'epochs': 40, 'optimizer': 'SGD'}
0.956089 (0.004658) with: {'batch_size': 16, 'epochs': 40, 'optimizer': 'Adam'}
0.955863 (0.004744) with: {'batch_size': 16, 'epochs': 50, 'optimizer': 'SGD'}
0.955863 (0.004903) with: {'batch_size': 16, 'epochs': 50, 'optimizer': 'Adam'}
0.956089 (0.004658) with: {'batch_size': 32, 'epochs': 10, 'optimizer': 'SGD'}
0.955863 (0.004744) with: {'batch_size': 32, 'epochs